In [ ]:
# !conda install -c stellargraph stellargraph

In [73]:
from scripts.tf_models import BaseTFModel
from tensorflow.keras import models, layers
from dgl.nn.tensorflow import GraphConv


In [ ]:
blocks = [1, 16, 32, 64, 32, 128]

In [24]:
import tensorflow as tf
from tensorflow.keras import layers
from dgl.nn.tensorflow import GraphConv

class GCN(tf.keras.Model):
    def __init__(self,
                 g,
                 in_feats,
                 n_hidden,
                 n_classes,
                 n_layers,
                 activation,
                 dropout):
        super(GCN, self).__init__()
        self.g = g
        self.layer_list = []
        # input layer
        self.layer_list.append(GraphConv(in_feats, n_hidden, activation=activation))
        # hidden layers
        for i in range(n_layers - 1):
            self.layer_list.append(GraphConv(n_hidden, n_hidden, activation=activation))
        # output layer
        self.layer_list.append(GraphConv(n_hidden, n_classes))
        self.dropout = layers.Dropout(dropout)

    def call(self, features):
        h = features
        for i, layer in enumerate(self.layer_list):
            if i != 0:
                h = self.dropout(h)
            h = layer(self.g, h)
        return h

In [33]:
gcn = GCN(G, in_feats=17, n_hidden=32, n_classes=18, n_layers=1, activation="relu", dropout=0.2)

In [34]:
gcn(train_df3)

InvalidArgumentError: Exception encountered when calling layer "graph_conv_6" (type GraphConv).

Incompatible shapes: [21079,5,18] vs. [5,1,1] [Op:Mul]

Call arguments received:
  • graph=Graph(num_nodes=5, num_edges=25,
      ndata_schemes={}
      edata_schemes={'weight': Scheme(shape=(), dtype=tf.float64)})
  • feat=tf.Tensor(shape=(21079, 5, 18), dtype=float32)
  • weight=None

25

In [48]:
G.number_of_edges() == G.edata["weight"].shape[0]

True

In [49]:
G.edata['_edge_weight'] = G.edata["weight"]

In [50]:
import dgl.function as fn

In [51]:
aggregate_fn = fn.u_mul_e('h', '_edge_weight', 'm')

In [59]:
n_edges = G.number_of_edges()
degs = tf.cast(tf.identity(G.in_degrees()), dtype=tf.float32)
norm = tf.math.pow(degs, -0.5)

In [63]:
norm = tf.where(tf.math.is_inf(norm), tf.zeros_like(norm), norm)
G.ndata['norm'] = tf.expand_dims(norm, -1)

In [71]:
import tensorflow as tf
from tensorflow.keras import layers
from dgl.nn.tensorflow import GraphConv

class GCN(tf.keras.Model):
    def __init__(self,
                 g,
                 in_feats,
                 n_hidden,
                 n_classes,
                 n_layers,
                 activation,
                 dropout):
        super(GCN, self).__init__()
        self.g = g
        self.layer_list = []
        # input layer
        self.layer_list.append(GraphConv(in_feats, n_hidden, activation=activation))
        # hidden layers
        for i in range(n_layers - 1):
            self.layer_list.append(GraphConv(n_hidden, n_hidden, activation=activation))
        # output layer
        self.layer_list.append(GraphConv(n_hidden, n_classes))
        self.dropout = layers.Dropout(dropout)

    def call(self, features):
        h = features
        for i, layer in enumerate(self.layer_list):
            if i != 0:
                h = self.dropout(h)
            h = layer(self.g, h)
        return h

In [72]:
model = GCN(G,
            in_feats=17,
            n_hidden=32,
            n_classes=5,
            n_layers=2,
            activation=tf.nn.relu,
            dropout=0.1)

In [70]:
loss_fcn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, epsilon=1e-8)

In [ ]:
with tf.GradientTape() as tape:
    logits = model(features)
    loss_value = loss_fcn(labels[train_mask], logits[train_mask])
    for weight in model.trainable_weights:
        loss_value = loss_value + args.weight_decay*tf.nn.l2_loss(weight)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

In [39]:
class Linear2(BaseTFModel):
    def __init__(self, window):
        super().__init__(window)

    def add_model(self, model):
        # temporal conv layer
        model.add(layers.Conv2D(16, kernel_size=(2,1), strides=1, data_format="channels_last", activation="relu", padding="same", dilation_rate=1))
        model.add(layers.LayerNormalization())
        model.add(GraphConv(in_feats=16, out_feats=32, activation="relu"))
        # model.add(layers.Dense(self.OUT_STEPS, kernel_initializer=tf.initializers.zeros()))
        
        # model.add(layers.Dense(self.OUT_STEPS, kernel_initializer=tf.initializers.zeros()))
        # model.add(layers.Dropout(0.2))
        return model
    
model = Linear2(window).model
model.summary()

TypeError: in user code:


    TypeError: tf__call() missing 1 required positional argument: 'feat'


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import subprocess; FOLDER_PATH = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')
import warnings; warnings.filterwarnings("ignore")
import sys, os; sys.path.append(os.path.dirname(f'{FOLDER_PATH}/utils')); sys.path.append(os.path.dirname(f'{FOLDER_PATH}/scripts'))

In [8]:
# os.environ["DGLBACKEND"] = "pytorch"

from utils.base_helpers import read_data, split_data, scale_data, expand_data
from utils.dgl_helpers import STGCN_WAVE
from utils.tf_helpers import WindowGenerator

import pandas as pd
import torch.nn as nn
import torch

import scipy.sparse as sp
import dgl
import numpy as np
import tensorflow as tf

ModuleNotFoundError: No module named 'dgl.nn.pytorch'

In [9]:
df, plants = read_data(generate_speed_angle=True, add_lagged=True, number_of_plants=5)
train_df, valid_df, test_df = split_data(df, train_ratio=0.8, valid_ratio=0.1)
train_df2, valid_df2, test_df2 = expand_data(train_df, valid_df, test_df)
train_df3, valid_df3, test_df3 = scale_data(train_df2, valid_df2, test_df2, plants, scaler="minmax")

Selected plants:
 [968, 969, 1484, 1507, 1518]
Train start and end dates:	 2019-01-26 03:00:00 	 2021-06-22 09:00:00
Validation start and end dates:	 2021-06-22 10:00:00 	 2021-10-10 04:00:00
Test start and end dates:	 2021-10-10 05:00:00 	 2022-01-27 23:00:00


In [15]:
import pandas as pd
import scipy.sparse as sp
import dgl

In [26]:
def generate_adjacency_matrix(df, threshold=0.5):    
    data = df.reset_index()[["forecast_dt", "rt_plant_id", "production"]]
    data = pd.pivot_table(data, index="forecast_dt", columns="rt_plant_id", values="production")

    A = abs(data.corr())
    A = A[A > threshold].fillna(0)
    return A

adj = generate_adjacency_matrix(train_df)
sp_mx = sp.coo_matrix(adj)
G = dgl.from_scipy(sp_mx, eweight_name="weight")

In [ ]:
blocks = [17, 64, 32, 64, 32, 128]
lr = 0.001
n_window = 2 * 24
n_ahead = 1
num_nodes = train_df.rt_plant_id.nunique()

In [ ]:
loss = nn.MSELoss()
# device=torch.device("cuda")
device=torch.device("cpu")
G = G.to(device)
model = STGCN_WAVE(blocks, n_window, num_nodes, G, device=device).to(device)

optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)

In [17]:
from utils.tf_helpers import WindowGenerator

OUT_STEPS = 24
INPUT_WIDTH = 30
SHIFT=0

window = WindowGenerator(
    train_df=train_df3, valid_df=valid_df3, test_df=test_df3, 
    columns=[col for col in train_df.columns if col != "rt_plant_id"],
    input_width=INPUT_WIDTH, label_width=OUT_STEPS, shift=SHIFT, label_columns=["production"])
window

Total window size: 30
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
Label indices: [ 6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
Label column name(s): ['production']

In [ ]:
train_iterator = list(window.train.as_numpy_iterator())
valid_iterator = list(window.valid.as_numpy_iterator())
test_iterator = list(window.test.as_numpy_iterator())

X_train = np.concatenate([_[0] for _ in train_iterator])
y_train = np.concatenate([_[1] for _ in train_iterator])
X_valid = np.concatenate([_[0] for _ in valid_iterator])
y_valid = np.concatenate([_[1] for _ in valid_iterator])
X_test = np.concatenate([_[0] for _ in test_iterator])
y_test = np.concatenate([_[1] for _ in test_iterator])

print(X_train.shape,  y_train.shape)
print(X_valid.shape,  y_valid.shape)
print(X_test.shape,  y_test.shape)

permute_ = (0,2,3,1)
permute_ = (0,3,1,2)

X_train = torch.from_numpy(X_train).permute(permute_)
y_train = torch.from_numpy(y_train).permute(permute_)
X_valid = torch.from_numpy(X_valid).permute(permute_)
y_valid = torch.from_numpy(y_valid).permute(permute_)
X_test = torch.from_numpy(X_test).permute(permute_)
y_test = torch.from_numpy(y_test).permute(permute_)

print(X_train.shape,  y_train.shape)
print(X_valid.shape,  y_valid.shape)
print(X_test.shape,  y_test.shape)

batch_size = 50
train_data = torch.utils.data.TensorDataset(X_train, y_train)
train_iter = torch.utils.data.DataLoader(train_data, batch_size)
valid_data = torch.utils.data.TensorDataset(X_valid, y_valid)
valid_iter = torch.utils.data.DataLoader(valid_data, batch_size)
test_data = torch.utils.data.TensorDataset(X_test, y_test)
test_iter = torch.utils.data.DataLoader(test_data, batch_size)

In [ ]:
save_path = "stgcnwavemodel.pt"
epochs = 5

def evaluate_model(model, loss, data_iter):
    model.eval()
    l_sum, n = 0.0, 0
    with torch.no_grad():
        for x, y in data_iter:
            y_pred = model(x).view(len(x), -1)
            l = loss(y_pred, y)
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        return l_sum / n


def evaluate_metric(model, data_iter, scaler):
    model.eval()
    with torch.no_grad():
        mae, mape, mse = [], [], []
        for x, y in data_iter:
            y = scaler.inverse_transform(y.cpu().numpy()).reshape(-1)
            y_pred = scaler.inverse_transform(model(x).view(len(x), -1).cpu().numpy()).reshape(-1)
            d = np.abs(y - y_pred)
            mae += d.tolist()
            mape += (d / y).tolist()
            mse += (d ** 2).tolist()
        MAE = np.array(mae).mean()
        MAPE = np.array(mape).mean()
        RMSE = np.sqrt(np.array(mse).mean())
        return MAE, MAPE, RMSE

In [ ]:
from time import time
min_val_loss = np.inf

for epoch in range(1, epochs + 1):
    l_sum, n = 0.0, 0
    model.train()
    t_ = time()
    for x, y in train_iter:
        y_pred = model(x).view(len(x), -1)
        l = loss(y_pred, y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        l_sum += l.item() * y.shape[0]
        n += y.shape[0]
    scheduler.step()
    val_loss = evaluate_model(model, loss, val_iter)
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        torch.save(model.state_dict(), save_path)
    print("epoch", epoch, ", time:", round(time() - t_, 2),  ", train loss:", l_sum / n, ", validation loss:", val_loss)

    

In [ ]:
# !pip install tensorflow

In [ ]:
OUT_STEPS = 24
INPUT_WIDTH = 30
SHIFT=0

window = WindowGenerator(
    train_df=train_df3, valid_df=valid_df3, test_df=test_df3, 
    columns=[col for col in train_df.columns if col != "rt_plant_id"],
    input_width=INPUT_WIDTH, label_width=OUT_STEPS, shift=SHIFT, label_columns=["production"])
window